In [1]:
%load_ext autoreload
%autoreload 2

import os
import sys
import fury
import pandas as pd
pd.set_option("display.max_rows", 120)
pd.set_option("display.max_columns", 120)

import seaborn as sns
sns.set_context('notebook')
sns.set(style="whitegrid", font_scale=1.5)
sns.despine()
sns.set_color_codes()

import matplotlib.pyplot as plt
plt.xlim(0, 1)
plt.ylim(0, None)
from matplotlib.ticker import MaxNLocator
%matplotlib inline
%config InlineBackend.figure_format = 'retina'
plt.rcParams['figure.figsize'] = [20, 5]

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

import logging
logging.basicConfig(level=logging.INFO, stream=sys.stdout)

import vtk
import numpy as np
import nibabel as nib
from six import iteritems
from scipy.stats import norm
from nibabel.affines import apply_affine
from vtk.util import numpy_support as ns
from dipy.tracking.streamline import values_from_volume
from dipy.io.streamline import save_vtk_streamlines

In [2]:
main_folder = '/Volumes/LaCieThree/Currarino/'
dirs = os.listdir(main_folder)

In [3]:
def load_nii(fname):
    img = nib.load(fname)
    return img.get_fdata(), img.affine

def read_vtk(filename):
    if filename.endswith('xml') or filename.endswith('vtp'):
        polydata_reader = vtk.vtkXMLPolyDataReader()
    else:
        polydata_reader = vtk.vtkPolyDataReader()

    polydata_reader.SetFileName(filename)
    polydata_reader.Update()

    polydata = polydata_reader.GetOutput()

    return vtkpolydata_to_tracts(polydata)

def vtkpolydata_to_tracts(polydata):
    result = {'lines': ns.vtk_to_numpy(polydata.GetLines().GetData()),
              'points': ns.vtk_to_numpy(polydata.GetPoints().GetData()),
              'numberOfLines': polydata.GetNumberOfLines()
             }

    data = {}
    if polydata.GetPointData().GetScalars():
        data['ActiveScalars'] = polydata.GetPointData().GetScalars().GetName()
    if polydata.GetPointData().GetVectors():
        data['ActiveVectors'] = polydata.GetPointData().GetVectors().GetName()
    if polydata.GetPointData().GetTensors():
        data['ActiveTensors'] = polydata.GetPointData().GetTensors().GetName()

    for i in range(polydata.GetPointData().GetNumberOfArrays()):
        array = polydata.GetPointData().GetArray(i)
        np_array = ns.vtk_to_numpy(array)
        if np_array.ndim == 1:
            np_array = np_array.reshape(len(np_array), 1)
        data[polydata.GetPointData().GetArrayName(i)] = np_array

    result['pointData'] = data

    tracts, data = vtkpolydata_dictionary_to_tracts_and_data(result)
    return tracts, data

def vtkpolydata_dictionary_to_tracts_and_data(dictionary):
    dictionary_keys = {'lines', 'points', 'numberOfLines'}
    if not dictionary_keys.issubset(dictionary):
        raise ValueError("Dictionary must have the keys lines and points" + repr(dictionary))

    tract_data = {}
    tracts = []

    lines = np.asarray(dictionary['lines']).squeeze()
    points = dictionary['points']

    actual_line_index = 0
    number_of_tracts = dictionary['numberOfLines']
    original_lines = []
    for _ in range(number_of_tracts):
        tracts.append(points[lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1]])
        original_lines.append(
            np.array(lines[actual_line_index + 1:actual_line_index + lines[actual_line_index] + 1], copy=True))
        actual_line_index += lines[actual_line_index] + 1

    if 'pointData' in dictionary:
        point_data_keys = [it[0] for it in iteritems(dictionary['pointData']) if isinstance(it[1], np.ndarray)]

        for k in point_data_keys:
            array_data = dictionary['pointData'][k]
            if k not in tract_data:
                tract_data[k] = [array_data[f] for f in original_lines]
            else:
                np.vstack(tract_data[k])
                tract_data[k].extend([array_data[f] for f in original_lines[-number_of_tracts:]])

    return tracts, tract_data

In [4]:
for d in os.listdir(main_folder):
    d = os.path.abspath(os.path.join(main_folder, d))
    if os.path.isdir(d) and not d.startswith('.'):
        l5 = None
        s1 = None
        s2 = None
        s3 = None
        s4 = None
        mask = None
        for file in os.listdir(d):
            current_path = os.path.abspath(os.path.join(d, file))
            
            if not file.startswith('.'):
                if '.vtk' in current_path.lower() and 'l5' in current_path.lower():
                    l5 = read_vtk(current_path)[0]
                elif '.vtk' in current_path.lower() and 's1' in current_path.lower():
                    s1 = read_vtk(current_path)[0]
                elif '.vtk' in current_path.lower() and 's2' in current_path.lower():
                    s2 = read_vtk(current_path)[0]
                elif '.vtk' in current_path.lower() and 's3' in current_path.lower():
                    s3 = read_vtk(current_path)[0]
                elif '.vtk' in current_path.lower() and 's4' in current_path.lower():
                    s4 = read_vtk(current_path)[0]
                elif '.nii' in current_path.lower() and 'test' in current_path.lower():
                    mask, affine = load_nii(current_path)
        if mask is not None:
            print(d)
            if l5:
                l5_cut = []
                mapping = values_from_volume(mask, l5, affine)
                for m, f in zip(mapping, l5):
                    current = []
                    m = np.array(m)>0
                    for m1, f1 in zip(m, f):
                        if m1:
                            current.append(f1.tolist())
                    if len(current) > 1:
                        l5_cut.append(np.asarray(current))

                save_vtk_streamlines(l5_cut, os.path.join(d, 'l5_cut.vtk'))
            if s1:
                s1_cut = []
                mapping = values_from_volume(mask, s1, affine)
                for m, f in zip(mapping, s1):
                    current = []
                    m = np.array(m)>0
                    for m1, f1 in zip(m, f):
                        if m1:
                            current.append(f1.tolist())
                    if len(current) > 1:
                        s1_cut.append(np.asarray(current))

                save_vtk_streamlines(s1_cut, os.path.join(d, 's1_cut.vtk'))

            if s2:
                s2_cut = []
                mapping = values_from_volume(mask, s2, affine)
                for m, f in zip(mapping, s2):
                    current = []
                    m = np.array(m)>0
                    for m1, f1 in zip(m, f):
                        if m1:
                            current.append(f1.tolist())
                    if len(current) > 1:
                        s2_cut.append(np.asarray(current))

                save_vtk_streamlines(s2_cut, os.path.join(d, 's2_cut.vtk'))

            if s3:
                s3_cut = []
                mapping = values_from_volume(mask, s3, affine)
                for m, f in zip(mapping, s3):
                    current = []
                    m = np.array(m)>0
                    for m1, f1 in zip(m, f):
                        if m1:
                            current.append(f1.tolist())
                    if len(current) > 1:
                        s3_cut.append(np.asarray(current))

                save_vtk_streamlines(s3_cut, os.path.join(d, 's3_cut.vtk'))

            if s4:
                s4_cut = []
                mapping = values_from_volume(mask, s4, affine)
                for m, f in zip(mapping, s4):
                    current = []
                    m = np.array(m)>0
                    for m1, f1 in zip(m, f):
                        if m1:
                            current.append(f1.tolist())
                    if len(current) > 1:
                        s4_cut.append(np.asarray(current))

                save_vtk_streamlines(s4_cut, os.path.join(d, 's4_cut.vtk'))


/Volumes/LaCieThree/Currarino/1-2-08
/Volumes/LaCieThree/Currarino/1-2-29
/Volumes/LaCieThree/Currarino/1-4-07
